In [1]:
import requests
import re
import json
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import get_from_table, table_to_df
from tmdbv3api import Person

In [67]:
film_id = 'f_0l5xU'
film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
print('Film ID is "{}", film title is "{}", and url is {}'.format(film_id, film_title, letterboxd_url))

Film ID is "f_0l5xU", film title is "Waltzing with Brando", and url is https://boxd.it/l5xU


In [76]:
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
r = requests.get(letterboxd_url)
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = requests.get(redirected_url)
soup = BeautifulSoup(r.content, 'lxml')
rating_dict = json.loads(soup.find('script', {'type':"application/ld+json"}).string.split('\n')[2]).get('aggregateRating')
try:
    rating_mean = rating_dict.get('ratingValue')
except:
    rating_mean = np.nan
try:
    rating_count = rating_dict.get('ratingCount')
except:
    rating_count = np.nan

In [81]:
import cloudscraper

scraper = cloudscraper.create_scraper()
response = scraper.get(letterboxd_url)

In [84]:
soup = BeautifulSoup(response.content, 'lxml')

In [85]:
soup

<!DOCTYPE html>
<html class="context-client-unknown no-mobile no-js" id="html" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="The story of how Marlon Brando plucked Bernard Judge, an obscure but idealistic Los Angeles architect from his stable existence and convinced him that he should build the world’s first ecologically perfect retreat on a tiny and uninhabitable Tahitian island." name="description"/>
<meta content="video.movie" property="og:type"/>
<meta content="https://letterboxd.com/film/waltzing-with-brando/" property="og:url"/>
<meta content="Waltzing with Brando (2024)" property="og:title"/>
<meta content="The story of how Marlon Brando plucked Bernard Judge, an obscure but idealistic Los Angeles architect from his stable existence and convinced him that he should build the world’s first ecologically perfect retreat on a tiny and uninhabitable Tahitian island.

In [70]:
film_url_title = get_from_table('FILM_URL_TITLE', film_id, 'FILM_URL_TITLE')
initial_url = 'https://letterboxd.com/film/{}/members/rated/.5-5/'.format(film_url_title)
r = requests.get(initial_url)
redirected_url = r.url
if initial_url != redirected_url:
    new_film_url_title = redirected_url.replace('https://letterboxd.com/film/', '').replace('/members/rated/.5-5/', '')
    film_url_title = new_film_url_title
soup = BeautifulSoup(r.content, 'lxml')
metrics_dict = {}
for i in ['members', 'fans', 'likes', 'reviews', 'lists']:
    href_str = '/film/{}/{}/'.format(film_url_title, i)
    try:
        metric_string = soup.find('a', {'class': 'tooltip', 'href':href_str}).get('title')
        metric = int(metric_string[:metric_string.find('\xa0')].replace(',', ''))
    except:
        metric = 0
    metrics_dict[i] = metric
metrics_dict

{'members': 152, 'fans': 0, 'likes': 20, 'reviews': 44, 'lists': 469}

'https://letterboxd.com/film/waltzing-with-brando/members/rated/.5-5/'

In [66]:
json.loads(soup.find('script', {'type':"application/ld+json"}).string.split('\n')[2]).keys()

dict_keys(['image', '@type', 'director', 'dateModified', 'productionCompany', 'releasedEvent', '@context', 'url', 'actors', 'dateCreated', 'name', 'genre', '@id', 'countryOfOrigin', 'aggregateRating'])

In [45]:
print(str(soup))

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Access denied | letterboxd.com used Cloudflare to restrict access</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/main.css" id="cf_styles-css" rel="stylesheet"/>
<script>
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feedback-success"),b=new XMLHttpRequest;a={event:"feedba

In [10]:
r = requests.get('https://letterboxd.com/film/{}/'.format(film_url_title))
soup = BeautifulSoup(r.content, 'lxml')

In [35]:
import re
[str(soup)[m.start()-20: m.start()+100] for m in re.finditer('esi', str(soup))]

['tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-1200-1200-675-675-crop-000000.jpg?v=e67c7e8',
 'tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-1200-1200-675-675-crop-000000.jpg?v=e67c7e8',
 'tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-1200-1200-675-675-crop-000000.jpg?v=e67c7e8',
 'tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-960-960-540-540-crop-000000.jpg?v=e67c7e851',
 'rando/availability/?esiAllowUser=true&amp;esiAllowCountry=true">\n<section class="watch-panel">\n<div class="header">\n<h3 ',
 'iAllowUser=true&amp;esiAllowCountry=true">\n<section class="watch-panel">\n<div class="header">\n<h3 class="title">Where to',
 'debar-user-actions/?esiAllowUser=true">\n</span>\n<li class="panel-sharing sharing-toggle js-actions-panel-sharing">\n<butt',
 'do/friend-activity/?esiAllowUser=true">\n</div>\n<div class="js-csi" data-on-load="film-news" data-src="/csi/film/

In [17]:
str(soup)[1067:1100]

'esized/sm/upload/4w/kk/98/d3/8uk1'

In [14]:
soup.find('a', {'class': 'has-icon icon-top250 icon-16 tooltip'})

,FILM_ID,TOP_250_POSITION
0,f_01NTU,1
1,f_01w9q,2
2,f_02auI,3
3,f_02axi,4
4,f_02aNq,5
...,...,...
245,f_02agG,246
246,f_01wwk,247
247,f_02aCs,248
248,f_0bXge,249


,FILM_ID,FILM_URL_TITLE,CREATED_AT
0,f_02BwG,evangelion-30-you-can-not-redo,2023-05-25 16:30:47.081296
1,f_01U16,blood-simple,2023-05-25 16:30:48.721797
2,f_026Ja,joint-security-area,2023-05-25 16:30:50.364336
3,f_029Mw,octopussy,2023-05-25 16:30:52.394624
4,f_02aZc,edward-scissorhands,2023-05-25 16:30:53.774337
...,...,...,...
18031,f_0eARi,ethel-ernest,2024-02-26 08:31:38.624868
18032,f_0AdVq,the-governesses,2024-02-26 08:52:36.414989
18033,f_0CepE,kung-fu-panda-4,2024-02-26 08:52:41.409430
18034,f_0FnHI,y2k-2024,2024-02-26 08:52:46.418584


,FILM_URL_TITLE,TOP_250,FILM_ID
0,harakiri,1,f_01NTU
1,come-and-see,2,f_01w9q
2,12-angry-men,3,f_02auI
3,seven-samurai,4,f_02axi
4,the-godfather-part-ii,5,f_02aNq
...,...,...,...
245,the-green-mile,246,f_02agG
246,the-life-and-death-of-colonel-blimp,247,f_01wwk
247,once-upon-a-time-in-america,248,f_02aCs
248,happy-hour-2015-1,249,f_0bXge


In [79]:
[x for x in soup.findAll('li', {'class': "poster-container numbered-list-item"})]

[<li class="poster-container numbered-list-item" data-object-id="filmListEntry:3528180420" data-object-name="list" data-owner-rating="10"> <div class="really-lazy-load poster film-poster film-poster-43015 linked-film-poster" data-cache-busting-key="85488461" data-film-id="43015" data-film-slug="harakiri" data-image-height="187" data-image-width="125" data-linked="linked" data-poster-url="/film/harakiri/image-150/" data-show-menu="true" data-target-link="/film/harakiri/" data-target-link-target=""> <img alt="Harakiri" class="image" height="187" src="https://s.ltrbxd.com/static/img/empty-poster-125.1ac65679.png" width="125"/> <span class="frame"><span class="frame-title"></span></span> </div> <p class="list-number">1</p> </li>,
 <li class="poster-container numbered-list-item" data-object-id="filmListEntry:3528180421" data-object-name="list" data-owner-rating="10"> <div class="really-lazy-load poster film-poster film-poster-36192 linked-film-poster" data-cache-busting-key="0308090d" data-

In [53]:
r = requests.get(letterboxd_url)
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = requests.get(redirected_url)    
soup = BeautifulSoup(r.content, 'lxml')
cast = soup.find('div', {'id': 'tab-cast'}).findAll('a')
cast_record = {
    'FILM_ID': [film_id]*len(cast),
    'PERSON_ID': [Person().search(x.text)[0].get('id') for x in cast],
    'CHARACTER': [x.get('title') for x in cast],
    'CAST_ORDER': range(len(cast)),
    'CREATED_AT': [datetime.now()]*len(cast)
    }

pd.DataFrame(cast_record)

,FILM_ID,PERSON_ID,CHARACTER,CAST_ORDER,CREATED_AT
0,f_0xzKI,1976054,Ella,0,2023-06-30 19:03:42.303859
1,f_0xzKI,21315,Christian,1,2023-06-30 19:03:42.303859


In [49]:
person.search('anna-zlokovic')[0].get('id')

1647240

In [41]:
[x.get('href') for x in crew]

['/director/anna-zlokovic/',
 '/producer/anna-zlokovic/',
 '/producer/alexander-familian/',
 '/producer/david-moore-5/',
 '/producer/dawn-fanning-moore/',
 '/writer/anna-zlokovic/',
 '/editor/alexander-familian/',
 '/cinematography/powell-robinson/',
 '/production-design/michelle-patterson/',
 '/set-decoration/william-sanders-4/',
 '/composer/nick-chuba/',
 '/sound/charles-moody/',
 '/costume-design/nadine-sondej-robinson/',
 '/makeup/nadine-sondej-robinson/']

In [55]:
crew = soup.find('div', {'id': 'tab-crew'}).findAll('a')


In [61]:
crew

[<a class="text-slug" href="/director/anna-zlokovic/">Anna Zlokovic</a>,
 <a class="text-slug" href="/producer/anna-zlokovic/">Anna Zlokovic</a>,
 <a class="text-slug" href="/producer/alexander-familian/">Alexander Familian</a>,
 <a class="text-slug" href="/producer/david-moore-5/">David Moore</a>,
 <a class="text-slug" href="/producer/dawn-fanning-moore/">Dawn Fanning Moore</a>,
 <a class="text-slug" href="/writer/anna-zlokovic/">Anna Zlokovic</a>,
 <a class="text-slug" href="/editor/alexander-familian/">Alexander Familian</a>,
 <a class="text-slug" href="/cinematography/powell-robinson/">Powell Robinson</a>,
 <a class="text-slug" href="/production-design/michelle-patterson/">Michelle Patterson</a>,
 <a class="text-slug" href="/set-decoration/william-sanders-4/">William Sanders</a>,
 <a class="text-slug" href="/composer/nick-chuba/">Nick Chuba</a>,
 <a class="text-slug" href="/sound/charles-moody/">Charles Moody</a>,
 <a class="text-slug" href="/costume-design/nadine-sondej-robinson/"

In [67]:
jobs = [x.get('href').split('/')[1] for x in crew]
names = [x.get('href').split('/')[2] for x in crew]


In [71]:
names

['anna-zlokovic',
 'anna-zlokovic',
 'alexander-familian',
 'david-moore-5',
 'dawn-fanning-moore',
 'anna-zlokovic',
 'alexander-familian',
 'powell-robinson',
 'michelle-patterson',
 'william-sanders-4',
 'nick-chuba',
 'charles-moody',
 'nadine-sondej-robinson',
 'nadine-sondej-robinson']

In [74]:
names[3][:-2]

'david-moore'

In [73]:
Person().search()

[{'adult': False, 'gender': 2, 'id': 59231, 'known_for_department': 'Acting', 'name': 'Joel David Moore', 'original_name': 'Joel David Moore', 'popularity': 10.066, 'profile_path': '/pZQ4un6v0EANFMAiJYEaGRsCFWE.jpg', 'known_for': [{'adult': False, 'backdrop_path': '/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg', 'id': 19995, 'title': 'Avatar', 'original_language': 'en', 'original_title': 'Avatar', 'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.', 'poster_path': '/kyeqWdyUXW608qlYkRqosgbbJyK.jpg', 'media_type': 'movie', 'genre_ids': [28, 12, 14, 878], 'popularity': 172.815, 'release_date': '2009-12-15', 'video': False, 'vote_average': 7.571, 'vote_count': 29320}, {'adult': False, 'backdrop_path': '/4pNlHx6ytdYBDs94PgcS0wQkbc4.jpg', 'id': 76600, 'title': 'Avatar: The Way of Water', 'original_language': 'en', 'original_title': 'Avatar: The Way of Water', 'overvie

In [ ]:
crew_record = {
    'FILM_ID': [film_id]*len(crew),
    'PERSON_ID': [x.text for x in crew],
    'JOB': [x.get('title') for x in crew],
    'CREATED_AT': [datetime.now()]*len(crew)
    }

pd.DataFrame(crew_record)

In [ ]:
og_url = soup.find('meta', {'property': 'og:url'}).get('content')

In [42]:
r = requests.get('https://letterboxd.com/film/{}/members/rated/.5-5/'.format(film_url_title))
r

<Response [200]>

In [43]:
soup = BeautifulSoup(r.content, 'lxml')
soup

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="en" class="ie6 lte9 lte8 lte7 lte6 no-js"> <![endif]--><!--[if IE 7 ]>    <html lang="en" class="ie7 lte9 lte8 lte7 no-js"> <![endif]--><!--[if IE 8 ]>    <html lang="en" class="ie8 lte9 lte8 no-js"> <![endif]--><!--[if IE 9 ]>    <html lang="en" class="ie9 lte9 no-js"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-mobile no-js" id="html" lang="en"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="Everyone who has watched The Big Sleep" name="description"/>
<meta content="https://letterboxd.com/film/the-big-sleep/members/rated/.5-5/" property="og:url"/>
<meta content="Everyone who has watched The Big Sleep" property="og:title"/>
<meta content="Everyone who has watched The Big Sleep" property="og:description"/>
<meta content="https://s.ltrbxd.com/static/img/default-share.e38c5d62.png" property="og:image"/>
<

In [44]:
metrics_dict = {}
for i in ['members', 'fans', 'likes', 'reviews', 'lists']:
    href_str = '/film/{}/{}/'.format(film_url_title, i)
    try:
        metric_string = soup.find('a', {'class': 'tooltip', 'href':href_str}).get('title')
        metric = int(metric_string[:metric_string.find('\xa0')].replace(',', ''))
    except:
        metric = 0
    metrics_dict[i] = metric

In [45]:
metrics_dict

{'members': 79686,
 'fans': 333,
 'likes': 19636,
 'reviews': 7586,
 'lists': 33275}

In [46]:
film_url_title = get_from_table('FILM_URL_TITLE', film_id, 'FILM_URL_TITLE')
r = requests.get('https://letterboxd.com/esi/film/{}/stats/'.format(film_url_title))
soup = BeautifulSoup(r.content, 'lxml')
try:
    top_250_status = int(soup.find('a', {'class': 'has-icon icon-top250 icon-16 tooltip'}).text)
except:
    top_250_status = None

In [47]:
top_250_status